In [ ]:
import scanpy as sc
import scanorama
import anndata
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=120, dpi_save=150, facecolor='white', color_map='tab20b')

In [ ]:
OUTPUT_PATH = "C:/Users/julia/Project/scanorama_output"

### SIMPLE integration process using sc.external.pp.scanorama_integrate two integrate the two samples ###

load in anndata objects, filtering, pp, hvg, regressing & scaling have been performed. They were frozen right after PCA performance.
Results stored in h5ad file, now loaded in:

In [ ]:
anndata_BLN = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_N/AnnData_storage/BL_N.h5ad")
anndata_BLC = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_C/AnnData_storage/BL_C.h5ad")
anndata_BLA = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_A/AnnData_storage/BL_A.h5ad")

In [ ]:
print('BL_N: ', anndata_BLN.shape)
print('BL_C: ', anndata_BLC.shape)
print('BL_A: ', anndata_BLA.shape)

make the obs 'batch' that is a dramatic overrepresentation of the sample name, because otherwise scanorama does not work :(

In [ ]:
anndata_BLA.obs['batch'] = 'BL_A'
anndata_BLC.obs['batch'] = 'BL_C'
anndata_BLN.obs['batch'] = 'BL_N'

In [ ]:
print('BL_N\n', anndata_BLN, '\nBL_C\n', anndata_BLC, '\nBL_A\n', anndata_BLA)

sc.external.pp.scanorama() does not directly accept a list of AnnData objects, you need to concatinate the objects firts, then yeet them in the function, and seperate them by key (batch in this case)

In [ ]:
combi = anndata.concat([anndata_BLC, anndata_BLA], index_unique="_")

Run Scanorma, the key 'batch' simply has the sample name stored.

In [ ]:
# default settings:
sc.external.pp.scanorama_integrate(combi, key='batch', basis='X_pca', adjusted_basis='X_scanorama', knn=5, sigma=15, approx=False, alpha=0.1, batch_size=5000)
# sc.external.pp.scanorama_integrate(combi, key='batch')

In [ ]:
combi

In [ ]:
adjusted_pcs = combi.obsm['X_scanorama']
combi.obsm['X_pca']=adjusted_pcs

In [ ]:
sc.pp.neighbors(combi)
sc.tl.leiden(combi)
sc.tl.umap(combi)

In [ ]:
sc.pl.umap(combi, color='batch', palette='tab20c',
             color_map='magma', title='BL A & BL C integrated UMAP',
             return_fig=False, show=False)

--------------------------------------------------------------------------------------------------------------------

Use Scanorama directly

In [ ]:
anndata_BLN = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_N/AnnData_storage/BL_N.h5ad")
anndata_BLC = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_C/AnnData_storage/BL_C.h5ad")
anndata_BLA = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_A/AnnData_storage/BL_A.h5ad")

In [ ]:
anndata_BLA.obs['batch'] = 'BL_A'
anndata_BLC.obs['batch'] = 'BL_C'
anndata_BLN.obs['batch'] = 'BL_N'

Note: HVG found that both sets have in common are the same as the concatinated dataset I made in the code above

In [ ]:
data_list = [anndata_BLA, anndata_BLC]
scanorama.integrate_scanpy(data_list)
# integrated_data = anndata.AnnData(X=integrated_data)


In [ ]:
print('BL_A:\n', data_list[0],'\nBL_C:\n', data_list[1])

In [ ]:
integrated_adata = anndata.AnnData(X=anndata.concat(data_list))

In [ ]:
integrated_adata

In [ ]:
sc.pp.neighbors(integrated_adata)
sc.tl.umap(integrated_adata)

In [ ]:
sc.pl.umap(integrated_adata, color='batch', 
           legend_loc='on data', palette='tab20b',
             color_map='magma', title='BL A & BL C integrated Scanorama UMAP',
             return_fig=False, show=False)

In [ ]:
integrated_adata

--------------------------------------------------------------------------------------------------------------------

Try out harmony wrapper for scanpy

In [ ]:
anndata_BLN = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_N/AnnData_storage/BL_N.h5ad")
anndata_BLC = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_C/AnnData_storage/BL_C.h5ad")
anndata_BLA = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_A/AnnData_storage/BL_A.h5ad")

In [ ]:
anndata_BLA.obs['batch'] = 'BL_A'
anndata_BLC.obs['batch'] = 'BL_C'
anndata_BLN.obs['batch'] = 'BL_N'

I think the results are all the same because of this line --> look into this more w/ Maurits

In [ ]:
combi = anndata.concat([anndata_BLC, anndata_BLN])

In [ ]:
sc.external.pp.harmony_integrate(combi, key='batch', basis='X_pca', adjusted_basis='X_pca_harmony')

In [ ]:
adjusted_pcs = combi.obsm['X_pca_harmony']
combi.obsm['X_pca']=adjusted_pcs

In [ ]:
sc.pp.neighbors(combi)
sc.tl.leiden(combi)
sc.tl.umap(combi)

In [ ]:
sc.pl.umap(combi, color='batch', palette='Set1',
             color_map='magma', title='BL N & BL C integrated harmony UMAP')

-------------------------------------------------------------------------------------------------------------------

Trying out bbknn() with the help of the vignettes

In [ ]:
import bbknn

In [ ]:
anndata_BLN = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_N/AnnData_storage/BL_N.h5ad")
anndata_BLC = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_C/AnnData_storage/BL_C.h5ad")
anndata_BLA = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_A/AnnData_storage/BL_A.h5ad")
anndata_BLA.obs['batch'] = 'BL_A'
anndata_BLC.obs['batch'] = 'BL_C'
anndata_BLN.obs['batch'] = 'BL_N'

In [ ]:
holder=[]
holder.append(anndata_BLC)
holder.append(anndata_BLN)

use concatenate according to tutorial (not concat()). This find more genes.
but concat() maintains sample names, in the cluster this looks nicer.

In [ ]:
adata = anndata.concat(holder)
#adata = holder[0].concatenate(holder[1:], join='outer')

concatinate() (older function) adds nr 0 - 1 instead of batch name. but 0 is BL_C and 1 is BL_N/BL_A.
concat() simply removes all variation and merges everything --> also looks for common hvg, takes way less hvg downstream than concatinate() does.
(6000 vs 1700)

In [ ]:
adata

In [ ]:
bbknn.bbknn(adata)

In [ ]:
adata

In [ ]:
sc.tl.umap(adata, neighbors_key='neighbors')
sc.tl.leiden(adata, resolution=1)

In [ ]:
adata.obs['batch']

In [ ]:
adata

In [ ]:
sc.pl.umap(adata, color=['leiden', 'batch'], palette='tab20b', title='First trial BL_C & BL_N bbknn package')

Now lets try to add ridge regression according to the github vignette
- 1st error was numpy error --> made issue in git now solved
- 2nd error is nan error after running bbknn --> this issue only arises when using concatinate() (OLD function) not with concat() so go with concat for now :)
NOTE: we don't necessary need this step, I just saw it in other vigenttes and wanted to try it

In [ ]:
bbknn.ridge_regression(adata, batch_key=['batch'], confounder_key='leiden')

idk what this x_explained layer is but it appears after running the ridge regression.
According to the vignette you should re-run pca, bbknn and also the umap

In [ ]:
adata.layers['X_explained']

In [ ]:
sc.pp.pca(adata)
bbknn.bbknn(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['batch', 'leiden'], palette='Set1', title='ridge regression in BL_C and BL_N')

In [ ]:
adata

let's try finding markergenes

In [ ]:
neurons = ['MAP2', 'DCX', 'NEUROG2', 'RBFOX3', 'SLC17A7']
astrocytes = ['VIM', 'S100B', 'SOX9', 'FABP7', 'SLC1A3']

In [ ]:
adata_DE =  adata.raw.to_adata()
adata_DE

In [ ]:
sc.tl.rank_genes_groups(adata_DE, 'leiden', method='wilcoxon', corr_method='bonferroni', key='wilcoxon', pts=True, )

In [ ]:
sc.tl.filter_rank_genes_groups(adata_DE, groupby='leiden', min_in_group_fraction=0.1, min_fold_change=1)

In [ ]:
with plt.rc_context({'figure.figsize': (3, 3)}):
    sc.pl.umap(adata_DE, color=astrocytes, s=50, frameon=False, ncols=4, vmax='p99', cmap='Reds', palette='Reds')

---------------------------------------------------------------------------------------------------------------------

### HARMONYPY ###
github: https://github.com/slowkow/harmonypy
code used from this issue: https://github.com/slowkow/harmonypy/issues/5


In [ ]:
import harmonypy as harm

load in anndata objects as normal and add 'batch' key

In [ ]:
anndata_BLN = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_N/AnnData_storage/BL_N.h5ad")
anndata_BLC = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_C/AnnData_storage/BL_C.h5ad")
anndata_BLA = sc.read_h5ad("C:/Users/julia/Project/ipynb_output/BL_A/AnnData_storage/BL_A.h5ad")
anndata_BLA.obs['batch'] = 'BL_A'
anndata_BLC.obs['batch'] = 'BL_C'
anndata_BLN.obs['batch'] = 'BL_N'

unfortunetaly their adata_all is once again a big anndata object with many batches, instead of multiple anndata's from different samples. I wonder if this tutorial will prove insightful

In [ ]:
holder=[]
holder.append(anndata_BLC)
holder.append(anndata_BLA)

In [ ]:
adata = anndata.concat(holder)
adata.obs_names_make_unique()

In [ ]:
data_mat = adata.obsm['X_pca']
meta_data = adata.obs
vars_use = ['batch']
ho = harm.run_harmony(data_mat, meta_data, vars_use)

In [ ]:
adjusted_pcs = pd.DataFrame(ho.Z_corr).T
adata.obsm['X_pca']=adjusted_pcs.values

In [ ]:
sc.pp.neighbors(adata)
sc.tl.leiden(adata)
sc.tl.umap(adata)


In [ ]:
sc.pl.umap(adata, color=['batch', 'leiden'], palette='tab20',
             color_map='magma', title='BL_C_BL_A_Harmonypy')

-----------------------------------------------------------------------------------------------------

Try combining all for fun

In [ ]:
holder=[]
holder.append(anndata_BLC)
holder.append(anndata_BLN)
holder.append(anndata_BLA)

In [ ]:
adata_all = anndata.concat(holder)

In [ ]:
sc.external.pp.bbknn(adata_all, batch_key='batch')

In [ ]:
sc.tl.umap(adata_all)
sc.tl.leiden(adata_all, resolution=1)

In [ ]:
adata_all

In [ ]:
sc.pl.umap(adata_all, color=['leiden', 'batch'], palette='tab20b', title='2nd trial all combined')

Using scanorama functions (not scanpy)

In [ ]:
# list = [anndata_BLN.raw.X, anndata_BLC.raw.X]
# genes_list = [anndata_BLN.raw.var_names, anndata_BLC.raw.var_names]

looks like concat, but does not return the same object. This seems to be going good. still finds 1762 genes in common (on hvg genes) and 18682 in common (on all raw genes)
this function gets called in correct

In [ ]:
# a, b = scanorama.merge_datasets(list, genes_list)

correct does a batch correction and integration at the same time?

In [ ]:
# datasets, genes = scanorama.correct(list, genes_list)

In [ ]:
# datasets

In [ ]:
# genes